In [1]:
%pip install fake_useragent

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import requests
import random
import time
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

In [3]:
ua = UserAgent()

In [4]:
def get_news(num: int) -> dict | None:
    url = f'https://www.moex.com/n{num}?nt=0?print=1'
    response = requests.get(url, headers={'User-Agent': ua.random})
    
    if response.status_code != 200:
        return None
    
    result = {}
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # date time
    div_datetime = soup.find('div', class_='news_date')
    if div_datetime:
        result['datetime'] = div_datetime.text
    
    # title
    h_title = soup.find('h1', class_='news_title')
    if h_title:
        result['title'] = h_title.text
    
    # text
    div_news_text = soup.find('div', class_='news_text')
    if div_news_text:
        # paragraphs = div_news_text.find_all('p')
        # merged_text = ""
        # for paragraph in paragraphs:
        #     merged_text += paragraph.text + ' '
        #     
        # result['text'] = merged_text
        result['text'] = div_news_text.text
    
    return result

In [ ]:
current = []
checkpoint = 0
for num in tqdm(range(68822, 33464, -1)):
    res = get_news(num)
    if not res:
        continue
    
    current.append([res['datetime'], res['title'], res['text']])
    if len(current) == 1000:
        checkpoint += 1
        df = pd.DataFrame(current, columns=['datetime', 'title', 'text'])
        df.to_csv(f'moex_data/part{checkpoint}.csv', index=False)
        current.clear()
        print(f'saved checkpoint: {checkpoint}')
    
    # time.sleep(random.randint(2, 5))

  3%|▎         | 1056/35358 [04:54<2:37:32,  3.63it/s]

saved checkpoint: 1


  6%|▌         | 2119/35358 [10:07<3:22:17,  2.74it/s]

saved checkpoint: 2


  9%|▉         | 3157/35358 [15:09<2:28:42,  3.61it/s]

...

 83%|████████▎ | 29245/35358 [2:17:26<26:08,  3.90it/s]  

saved checkpoint: 23


 98%|█████████▊| 34489/35358 [2:43:05<04:05,  3.53it/s]

saved checkpoint: 28


 98%|█████████▊| 34498/35358 [2:43:08<03:39,  3.92it/s]

In [4]:
df = pd.DataFrame()
for i in tqdm(range(1, 29)):
    df = pd.concat([df, pd.read_csv(f'moex_data/part{i}.csv')], ignore_index=True)
df.head()

100%|██████████| 28/28 [00:02<00:00, 11.53it/s]


,datetime,title,text
0,05.04.2024 17:45,О значениях индикативных ставок риска на фондо...,\nИнформируем о значении Индикативных ставок р...
1,05.04.2024 17:32,О допуске ценных бумаг к операциям РЕПО с ЦК с...,"\nУчастникам торгов\nИнформируем Вас о том, чт..."
2,05.04.2024 17:16,Maintenance on T1 securities and FX test envir...,\nFrom 8 to 10 April 2024 we will be updating ...
3,05.04.2024 17:16,Технические работы на тестовом контуре T1 фонд...,\nС 8 по 10 апреля 2024 года на выделенном тес...
4,05.04.2024 17:13,Risk parameters change for the security RU000A...,\r\n As per the Securities market r...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  28000 non-null  object
 1   title     28000 non-null  object
 2   text      28000 non-null  object
dtypes: object(3)
memory usage: 656.4+ KB
